```
================================================================================
PROYECTO FINAL - INTELIGENCIA ARTIFICIAL
SECCIÓN 4: APRENDIZAJE SUPERVISADO (TAREAS 13, 14, 15, 16, 17)
================================================================================
Universidad del Norte - Ingeniería de Sistemas
Profesor: Eduardo Zurek, Ph.D.

OBJETIVO: Entrenar y evaluar modelos de clasificación supervisada para predecir
el nivel de desempeño en inglés (DESEMP_INGLES) a partir de las variables
preprocesadas.
================================================================================
```

In [ ]:
# ============================================
# CONFIGURACIÓN INICIAL E IMPORTACIONES
# ============================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle
import json
from time import time

# Scikit-learn imports - Modelos
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

# Métricas y validación
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score,
    roc_curve, precision_recall_curve, matthews_corrcoef
)
from sklearn.model_selection import (
    cross_val_score, cross_validate, StratifiedKFold,
    GridSearchCV, RandomizedSearchCV
)

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

RANDOM_STATE = 42
TARGET_COLUMN = 'DESEMP_INGLES'
np.random.seed(RANDOM_STATE)

print("="*80)
print("SECCIÓN 4: APRENDIZAJE SUPERVISADO")
print("="*80)
print(f"📌 Random State: {RANDOM_STATE}")
print(f"🎯 Variable Objetivo: {TARGET_COLUMN}\n")

In [ ]:
# ============================================
# CARGAR DATOS PREPROCESADOS
# ============================================

print("="*80)
print("CARGANDO DATOS PREPROCESADOS DE SECCIÓN 2")
print("="*80 + "\n")

try:
    # Cargar train/test split
    with open('train_test_split.pkl', 'rb') as f:
        train_test_data = pickle.load(f)
    
    X_train = train_test_data['X_train']
    X_test = train_test_data['X_test']
    y_train = train_test_data['y_train']
    y_test = train_test_data['y_test']
    y_train_original = train_test_data['y_train_original']
    y_test_original = train_test_data['y_test_original']
    
    print(f"✅ X_train: {X_train.shape}")
    print(f"✅ X_test: {X_test.shape}")
    print(f"✅ y_train: {y_train.shape}")
    print(f"✅ y_test: {y_test.shape}")
    
    # Cargar objetos de preprocesamiento
    with open('preprocessing_objects.pkl', 'rb') as f:
        preprocessing_objects = pickle.load(f)
    
    le_target = preprocessing_objects['label_encoder_target']
    target_mapping = preprocessing_objects['target_mapping']
    final_features = preprocessing_objects['final_features']
    
    print(f"\n✅ Mapeo de clases: {target_mapping}")
    print(f"✅ Features: {len(final_features)}")
    
    # Cargar PCA (opcional)
    try:
        with open('pca_models.pkl', 'rb') as f:
            pca_objects = pickle.load(f)
        X_train_pca = pca_objects['X_train_pca']
        X_test_pca = pca_objects['X_test_pca']
        optimal_n_components = pca_objects['optimal_n_components']
        print(f"✅ PCA disponible: {optimal_n_components} componentes")
    except:
        X_train_pca = None
        X_test_pca = None
        print("⚠️  PCA no disponible")
        
except Exception as e:
    print(f"❌ Error cargando datos: {e}")
    print("⚠️  Ejecuta primero la Sección 2 (Preprocesamiento)")
    raise

# Verificar distribución de clases
print("\n📊 DISTRIBUCIÓN DE CLASES:")
for i, label in enumerate(le_target.classes_):
    n_train = np.sum(y_train == i)
    n_test = np.sum(y_test == i)
    pct_train = n_train / len(y_train) * 100
    pct_test = n_test / len(y_test) * 100
    print(f"   {label}: Train={n_train:,} ({pct_train:.1f}%), Test={n_test:,} ({pct_test:.1f}%)")

print("\n✅ Datos cargados correctamente")

In [ ]:
# ============================================
# TAREA 13: ENTRENAMIENTO DE MODELOS
# ============================================

print("\n\n" + "="*80)
print("TAREA 13: ENTRENAMIENTO DE MODELOS DE CLASIFICACIÓN")
print("="*80 + "\n")

"""
Entrenaremos múltiples modelos de clasificación:
1. Decision Tree - Modelo interpretable basado en reglas
2. Random Forest - Ensemble de árboles de decisión
3. Logistic Regression - Modelo lineal probabilístico
4. Support Vector Machine (SVM) - Modelo de margen máximo
5. K-Nearest Neighbors (KNN) - Modelo basado en instancias
"""

# Diccionario para almacenar modelos y resultados
models = {}
predictions = {}
training_times = {}

# ==================
# MODELO 1: DECISION TREE
# ==================

print("13.1 DECISION TREE CLASSIFIER")
print("-" * 80 + "\n")

print("Configuración:")
print("   • max_depth = 15")
print("   • min_samples_split = 100")
print("   • min_samples_leaf = 50")
print("   • criterion = 'gini'")

start_time = time()
dt_model = DecisionTreeClassifier(
    max_depth=15,
    min_samples_split=100,
    min_samples_leaf=50,
    criterion='gini',
    random_state=RANDOM_STATE
)
dt_model.fit(X_train, y_train)
dt_time = time() - start_time

models['Decision Tree'] = dt_model
predictions['Decision Tree'] = dt_model.predict(X_test)
training_times['Decision Tree'] = dt_time

print(f"\n✅ Decision Tree entrenado en {dt_time:.2f}s")
print(f"   • Número de nodos: {dt_model.tree_.node_count}")
print(f"   • Profundidad del árbol: {dt_model.tree_.max_depth}")

# ==================
# MODELO 2: RANDOM FOREST
# ==================

print("\n\n13.2 RANDOM FOREST CLASSIFIER")
print("-" * 80 + "\n")

print("Configuración:")
print("   • n_estimators = 100")
print("   • max_depth = 15")
print("   • min_samples_split = 50")
print("   • min_samples_leaf = 25")

start_time = time()
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=15,
    min_samples_split=50,
    min_samples_leaf=25,
    random_state=RANDOM_STATE,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)
rf_time = time() - start_time

models['Random Forest'] = rf_model
predictions['Random Forest'] = rf_model.predict(X_test)
training_times['Random Forest'] = rf_time

print(f"\n✅ Random Forest entrenado en {rf_time:.2f}s")
print(f"   • Número de árboles: {rf_model.n_estimators}")

# ==================
# MODELO 3: LOGISTIC REGRESSION
# ==================

print("\n\n13.3 LOGISTIC REGRESSION")
print("-" * 80 + "\n")

print("Configuración:")
print("   • solver = 'lbfgs'")
print("   • max_iter = 1000")
print("   • multi_class = 'multinomial'")

start_time = time()
lr_model = LogisticRegression(
    solver='lbfgs',
    max_iter=1000,
    multi_class='multinomial',
    random_state=RANDOM_STATE,
    n_jobs=-1
)
lr_model.fit(X_train, y_train)
lr_time = time() - start_time

models['Logistic Regression'] = lr_model
predictions['Logistic Regression'] = lr_model.predict(X_test)
training_times['Logistic Regression'] = lr_time

print(f"\n✅ Logistic Regression entrenado en {lr_time:.2f}s")

# ==================
# MODELO 4: SUPPORT VECTOR MACHINE
# ==================

print("\n\n13.4 SUPPORT VECTOR MACHINE (SVM)")
print("-" * 80 + "\n")

print("Configuración:")
print("   • kernel = 'rbf'")
print("   • C = 1.0")
print("   • gamma = 'scale'")
print("⚠️  Usando muestra de 20,000 observaciones por eficiencia...\n")

# SVM es costoso, usar muestra
sample_indices = np.random.choice(len(X_train), min(20000, len(X_train)), replace=False)
X_train_svm = X_train.iloc[sample_indices]
y_train_svm = y_train.iloc[sample_indices]

start_time = time()
svm_model = SVC(
    kernel='rbf',
    C=1.0,
    gamma='scale',
    random_state=RANDOM_STATE
)
svm_model.fit(X_train_svm, y_train_svm)
svm_time = time() - start_time

models['SVM'] = svm_model
predictions['SVM'] = svm_model.predict(X_test)
training_times['SVM'] = svm_time

print(f"✅ SVM entrenado en {svm_time:.2f}s")
print(f"   • Support vectors: {svm_model.n_support_.sum()}")

# ==================
# MODELO 5: K-NEAREST NEIGHBORS
# ==================

print("\n\n13.5 K-NEAREST NEIGHBORS (KNN)")
print("-" * 80 + "\n")

print("Configuración:")
print("   • n_neighbors = 7")
print("   • weights = 'distance'")
print("   • metric = 'euclidean'")

start_time = time()
knn_model = KNeighborsClassifier(
    n_neighbors=7,
    weights='distance',
    metric='euclidean',
    n_jobs=-1
)
knn_model.fit(X_train, y_train)
knn_time = time() - start_time

models['KNN'] = knn_model
predictions['KNN'] = knn_model.predict(X_test)
training_times['KNN'] = knn_time

print(f"\n✅ KNN entrenado en {knn_time:.2f}s")

# Resumen
print("\n\n" + "="*80)
print("RESUMEN DE ENTRENAMIENTO")
print("="*80 + "\n")

for model_name, t in training_times.items():
    print(f"   {model_name:20s}: {t:7.2f}s")

print("\n" + "="*80)
print("✅ TAREA 13 COMPLETADA")
print("="*80)

In [ ]:
# ============================================
# TAREA 14: COMPARACIÓN DE MODELOS
# ============================================

print("\n\n" + "="*80)
print("TAREA 14: COMPARACIÓN DE MODELOS CON MÉTRICAS")
print("="*80 + "\n")

# Calcular métricas para cada modelo
metrics_results = {}

for model_name in models.keys():
    y_pred = predictions[model_name]
    
    # Calcular métricas
    acc = accuracy_score(y_test, y_pred)
    # Para multi-clase, usar 'weighted' average
    prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    metrics_results[model_name] = {
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1-Score': f1,
        'Training Time': training_times[model_name]
    }

# Crear DataFrame de comparación
metrics_df = pd.DataFrame(metrics_results).T
metrics_df = metrics_df.sort_values('F1-Score', ascending=False)

print("📊 TABLA COMPARATIVA DE MÉTRICAS:\n")
print(metrics_df.to_string())

# Visualización de comparación
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Accuracy comparison
metrics_df['Accuracy'].plot(kind='barh', ax=axes[0, 0], color='skyblue')
axes[0, 0].set_xlabel('Accuracy', fontweight='bold')
axes[0, 0].set_title('Comparación de Accuracy', fontsize=13, fontweight='bold')
axes[0, 0].grid(axis='x', alpha=0.3)
for i, v in enumerate(metrics_df['Accuracy']):
    axes[0, 0].text(v + 0.005, i, f'{v:.4f}', va='center')

# 2. F1-Score comparison
metrics_df['F1-Score'].plot(kind='barh', ax=axes[0, 1], color='lightcoral')
axes[0, 1].set_xlabel('F1-Score', fontweight='bold')
axes[0, 1].set_title('Comparación de F1-Score', fontsize=13, fontweight='bold')
axes[0, 1].grid(axis='x', alpha=0.3)
for i, v in enumerate(metrics_df['F1-Score']):
    axes[0, 1].text(v + 0.005, i, f'{v:.4f}', va='center')

# 3. Precision vs Recall
axes[1, 0].scatter(metrics_df['Precision'], metrics_df['Recall'], s=200, alpha=0.6)
for idx, model in enumerate(metrics_df.index):
    axes[1, 0].annotate(model, 
                       (metrics_df['Precision'].iloc[idx], metrics_df['Recall'].iloc[idx]),
                       fontsize=9, ha='center')
axes[1, 0].set_xlabel('Precision', fontweight='bold')
axes[1, 0].set_ylabel('Recall', fontweight='bold')
axes[1, 0].set_title('Precision vs Recall', fontsize=13, fontweight='bold')
axes[1, 0].grid(alpha=0.3)

# 4. Training Time
metrics_df['Training Time'].plot(kind='barh', ax=axes[1, 1], color='lightgreen')
axes[1, 1].set_xlabel('Tiempo (segundos)', fontweight='bold')
axes[1, 1].set_title('Tiempo de Entrenamiento', fontsize=13, fontweight='bold')
axes[1, 1].grid(axis='x', alpha=0.3)
for i, v in enumerate(metrics_df['Training Time']):
    axes[1, 1].text(v + 0.5, i, f'{v:.2f}s', va='center')

plt.tight_layout()
plt.savefig('model_comparison_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

# Matrices de confusión
print("\n\n14.2 MATRICES DE CONFUSIÓN\n")
print("-" * 80 + "\n")

n_models = len(models)
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for idx, (model_name, y_pred) in enumerate(predictions.items()):
    cm = confusion_matrix(y_test, y_pred)
    
    # Normalizar por fila (porcentajes)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues', 
                xticklabels=le_target.classes_,
                yticklabels=le_target.classes_,
                ax=axes[idx], cbar=True)
    
    axes[idx].set_title(f'{model_name}\n(Acc: {metrics_results[model_name]["Accuracy"]:.4f})', 
                       fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('Real', fontweight='bold')
    axes[idx].set_xlabel('Predicho', fontweight='bold')

# Hide last subplot if odd number of models
if n_models < 6:
    axes[5].axis('off')

plt.tight_layout()
plt.savefig('confusion_matrices.png', dpi=300, bbox_inches='tight')
plt.show()

# Classification reports
print("\n14.3 REPORTES DE CLASIFICACIÓN DETALLADOS\n")
print("-" * 80 + "\n")

for model_name, y_pred in predictions.items():
    print(f"\n{'='*80}")
    print(f"MODELO: {model_name}")
    print(f"{'='*80}\n")
    print(classification_report(y_test, y_pred, 
                                target_names=le_target.classes_,
                                digits=4))

print("\n" + "="*80)
print("✅ TAREA 14 COMPLETADA")
print("="*80)

In [ ]:
# ============================================
# TAREA 15: VALIDACIÓN CRUZADA
# ============================================

print("\n\n" + "="*80)
print("TAREA 15: VALIDACIÓN CRUZADA Y ANÁLISIS DE ESTABILIDAD")
print("="*80 + "\n")

print("15.1 CONFIGURACIÓN DE K-FOLD CROSS-VALIDATION")
print("-" * 80 + "\n")

# Configurar k-fold estratificado
N_FOLDS = 5
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)

print(f"Configuración: {N_FOLDS}-Fold Stratified Cross-Validation")
print(f"   • Número de folds: {N_FOLDS}")
print(f"   • Estratificación: Sí")
print(f"   • Shuffle: Sí\n")

# Métricas a evaluar
scoring = {
    'accuracy': 'accuracy',
    'precision_weighted': 'precision_weighted',
    'recall_weighted': 'recall_weighted',
    'f1_weighted': 'f1_weighted'
}

# Realizar cross-validation para cada modelo
cv_results = {}

print("15.2 EJECUTANDO VALIDACIÓN CRUZADA\n")
print("-" * 80 + "\n")

for model_name, model in models.items():
    print(f"Evaluando {model_name}...")
    
    # Para SVM, usar muestra por eficiencia
    if model_name == 'SVM':
        X_cv = X_train_svm
        y_cv = y_train_svm
    else:
        X_cv = X_train
        y_cv = y_train
    
    start_time = time()
    cv_scores = cross_validate(
        model, X_cv, y_cv, 
        cv=skf, 
        scoring=scoring,
        n_jobs=-1,
        return_train_score=True
    )
    cv_time = time() - start_time
    
    # Almacenar resultados
    cv_results[model_name] = {
        'accuracy_train': cv_scores['train_accuracy'],
        'accuracy_test': cv_scores['test_accuracy'],
        'precision_test': cv_scores['test_precision_weighted'],
        'recall_test': cv_scores['test_recall_weighted'],
        'f1_test': cv_scores['test_f1_weighted'],
        'time': cv_time
    }
    
    print(f"   ✅ Completado en {cv_time:.2f}s\n")

# Crear DataFrame de resultados
print("\n15.3 RESULTADOS DE VALIDACIÓN CRUZADA\n")
print("-" * 80 + "\n")

cv_summary = []
for model_name, scores in cv_results.items():
    cv_summary.append({
        'Modelo': model_name,
        'Accuracy (mean)': scores['accuracy_test'].mean(),
        'Accuracy (std)': scores['accuracy_test'].std(),
        'F1-Score (mean)': scores['f1_test'].mean(),
        'F1-Score (std)': scores['f1_test'].std(),
        'Train-Test Gap': scores['accuracy_train'].mean() - scores['accuracy_test'].mean()
    })

cv_summary_df = pd.DataFrame(cv_summary).sort_values('F1-Score (mean)', ascending=False)
print(cv_summary_df.to_string(index=False))

# Visualización de estabilidad
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Box plot de accuracy por fold
data_accuracy = [cv_results[name]['accuracy_test'] for name in models.keys()]
bp1 = axes[0, 0].boxplot(data_accuracy, labels=models.keys(), patch_artist=True)
for patch in bp1['boxes']:
    patch.set_facecolor('lightblue')
axes[0, 0].set_ylabel('Accuracy', fontweight='bold')
axes[0, 0].set_title(f'Distribución de Accuracy ({N_FOLDS}-Fold CV)', 
                     fontsize=13, fontweight='bold')
axes[0, 0].grid(axis='y', alpha=0.3)
axes[0, 0].tick_params(axis='x', rotation=45)

# 2. Box plot de F1-Score por fold
data_f1 = [cv_results[name]['f1_test'] for name in models.keys()]
bp2 = axes[0, 1].boxplot(data_f1, labels=models.keys(), patch_artist=True)
for patch in bp2['boxes']:
    patch.set_facecolor('lightcoral')
axes[0, 1].set_ylabel('F1-Score', fontweight='bold')
axes[0, 1].set_title(f'Distribución de F1-Score ({N_FOLDS}-Fold CV)', 
                     fontsize=13, fontweight='bold')
axes[0, 1].grid(axis='y', alpha=0.3)
axes[0, 1].tick_params(axis='x', rotation=45)

# 3. Train-Test Gap (overfitting indicator)
gap_data = {name: scores['accuracy_train'].mean() - scores['accuracy_test'].mean() 
            for name, scores in cv_results.items()}
axes[1, 0].barh(list(gap_data.keys()), list(gap_data.values()), color='orange', alpha=0.7)
axes[1, 0].set_xlabel('Train-Test Accuracy Gap', fontweight='bold')
axes[1, 0].set_title('Indicador de Overfitting\n(Menor es mejor)', 
                     fontsize=13, fontweight='bold')
axes[1, 0].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[1, 0].grid(axis='x', alpha=0.3)
for i, (k, v) in enumerate(gap_data.items()):
    axes[1, 0].text(v + 0.002, i, f'{v:.4f}', va='center')

# 4. Std deviation (stability indicator)
std_data = {name: scores['accuracy_test'].std() for name, scores in cv_results.items()}
axes[1, 1].barh(list(std_data.keys()), list(std_data.values()), color='lightgreen', alpha=0.7)
axes[1, 1].set_xlabel('Desviación Estándar (Accuracy)', fontweight='bold')
axes[1, 1].set_title('Estabilidad del Modelo\n(Menor es más estable)', 
                     fontsize=13, fontweight='bold')
axes[1, 1].grid(axis='x', alpha=0.3)
for i, (k, v) in enumerate(std_data.items()):
    axes[1, 1].text(v + 0.0002, i, f'{v:.5f}', va='center')

plt.tight_layout()
plt.savefig('cross_validation_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n" + "="*80)
print("✅ TAREA 15 COMPLETADA")
print("="*80)

print("\n💡 INTERPRETACIÓN:")
print("   • Menor desviación estándar = Mayor estabilidad")
print("   • Menor Train-Test Gap = Menos overfitting")
print("   • Mejor modelo: Alto F1, baja desviación, bajo gap")

In [ ]:
# ============================================# TAREA 16: AJUSTE DE HIPERPARÁMETROS# ============================================print("\n\n" + "="*80)print("TAREA 16: AJUSTE DE HIPERPARÁMETROS")print("="*80 + "\n")"""Realizaremos Grid Search para los 2 mejores modelos de la Task 15.Para otros modelos, usaremos Random Search para mayor eficiencia."""# Seleccionar los 2 mejores modelos según CVbest_models = cv_summary_df.head(2)['Modelo'].tolist()print(f"Modelos seleccionados para tuning: {best_models}\n")tuned_models = {}best_params_dict = {}# ==================# RANDOM FOREST - GRID SEARCH# ==================if 'Random Forest' in best_models:    print("16.1 RANDOM FOREST - GRID SEARCH")    print("-" * 80 + "\n")        param_grid_rf = {        'n_estimators': [50, 100, 150],        'max_depth': [10, 15, 20],        'min_samples_split': [50, 100, 150],        'min_samples_leaf': [25, 50, 75]    }        print("Grid de parámetros:")    for param, values in param_grid_rf.items():        print(f"   • {param}: {values}")        print(f"\nTotal de combinaciones: {np.prod([len(v) for v in param_grid_rf.values()])}")    print("Ejecutando Grid Search...\n")        start_time = time()    grid_rf = GridSearchCV(        RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1),        param_grid_rf,        cv=3,  # 3-fold para eficiencia        scoring='f1_weighted',        n_jobs=-1,        verbose=1    )    grid_rf.fit(X_train, y_train)    rf_tuning_time = time() - start_time        tuned_models['Random Forest (Tuned)'] = grid_rf.best_estimator_    best_params_dict['Random Forest'] = grid_rf.best_params_        print(f"\n✅ Grid Search completado en {rf_tuning_time:.2f}s")    print(f"\nMejores parámetros:")    for param, value in grid_rf.best_params_.items():        print(f"   • {param}: {value}")    print(f"\nMejor F1-Score (CV): {grid_rf.best_score_:.4f}")# ==================# LOGISTIC REGRESSION - GRID SEARCH  # ==================if 'Logistic Regression' in best_models:    print("\n\n16.2 LOGISTIC REGRESSION - GRID SEARCH")    print("-" * 80 + "\n")        param_grid_lr = {        'C': [0.001, 0.01, 0.1, 1.0, 10.0],        'solver': ['lbfgs', 'saga'],        'max_iter': [1000, 2000]    }        print("Grid de parámetros:")    for param, values in param_grid_lr.items():        print(f"   • {param}: {values}")        print(f"\nTotal de combinaciones: {np.prod([len(v) for v in param_grid_lr.values()])}")    print("Ejecutando Grid Search...\n")        start_time = time()    grid_lr = GridSearchCV(        LogisticRegression(random_state=RANDOM_STATE, multi_class='multinomial', n_jobs=-1),        param_grid_lr,        cv=3,        scoring='f1_weighted',        n_jobs=-1,        verbose=1    )    grid_lr.fit(X_train, y_train)    lr_tuning_time = time() - start_time        tuned_models['Logistic Regression (Tuned)'] = grid_lr.best_estimator_    best_params_dict['Logistic Regression'] = grid_lr.best_params_        print(f"\n✅ Grid Search completado en {lr_tuning_time:.2f}s")    print(f"\nMejores parámetros:")    for param, value in grid_lr.best_params_.items():        print(f"   • {param}: {value}")    print(f"\nMejor F1-Score (CV): {grid_lr.best_score_:.4f}")# ==================# DECISION TREE - RANDOM SEARCH# ==================print("\n\n16.3 DECISION TREE - RANDOM SEARCH")print("-" * 80 + "\n")from scipy.stats import randintparam_dist_dt = {    'max_depth': randint(5, 30),    'min_samples_split': randint(50, 200),    'min_samples_leaf': randint(20, 100),    'criterion': ['gini', 'entropy']}print("Distribución de parámetros:")for param, dist in param_dist_dt.items():    print(f"   • {param}: {dist}")print(f"\nIteraciones: 20")print("Ejecutando Random Search...\n")start_time = time()random_dt = RandomizedSearchCV(    DecisionTreeClassifier(random_state=RANDOM_STATE),    param_dist_dt,    n_iter=20,    cv=3,    scoring='f1_weighted',    random_state=RANDOM_STATE,    n_jobs=-1,    verbose=1)random_dt.fit(X_train, y_train)dt_tuning_time = time() - start_timetuned_models['Decision Tree (Tuned)'] = random_dt.best_estimator_best_params_dict['Decision Tree'] = random_dt.best_params_print(f"\n✅ Random Search completado en {dt_tuning_time:.2f}s")print(f"\nMejores parámetros:")for param, value in random_dt.best_params_.items():    print(f"   • {param}: {value}")print(f"\nMejor F1-Score (CV): {random_dt.best_score_:.4f}")# ==================# COMPARACIÓN: ANTES VS DESPUÉS# ==================print("\n\n16.4 COMPARACIÓN: MODELOS ORIGINALES VS OPTIMIZADOS")print("-" * 80 + "\n")comparison_results = []for model_name in best_params_dict.keys():    # Modelo original    original_pred = predictions[model_name]    original_f1 = f1_score(y_test, original_pred, average='weighted')    original_acc = accuracy_score(y_test, original_pred)        # Modelo optimizado    tuned_model = tuned_models[f'{model_name} (Tuned)']    tuned_pred = tuned_model.predict(X_test)    tuned_f1 = f1_score(y_test, tuned_pred, average='weighted')    tuned_acc = accuracy_score(y_test, tuned_pred)        comparison_results.append({        'Modelo': model_name,        'F1 (Original)': original_f1,        'F1 (Tuned)': tuned_f1,        'F1 Mejora': tuned_f1 - original_f1,        'Acc (Original)': original_acc,        'Acc (Tuned)': tuned_acc,        'Acc Mejora': tuned_acc - original_acc    })comparison_df = pd.DataFrame(comparison_results)print(comparison_df.to_string(index=False))# Visualizaciónfig, axes = plt.subplots(1, 2, figsize=(16, 6))# F1-Score comparisonx = np.arange(len(comparison_df))width = 0.35axes[0].bar(x - width/2, comparison_df['F1 (Original)'], width, label='Original', alpha=0.8)axes[0].bar(x + width/2, comparison_df['F1 (Tuned)'], width, label='Optimizado', alpha=0.8)axes[0].set_xlabel('Modelo', fontweight='bold')axes[0].set_ylabel('F1-Score', fontweight='bold')axes[0].set_title('Comparación F1-Score: Original vs Optimizado', fontsize=13, fontweight='bold')axes[0].set_xticks(x)axes[0].set_xticklabels(comparison_df['Modelo'], rotation=45, ha='right')axes[0].legend()axes[0].grid(axis='y', alpha=0.3)# Accuracy comparisonaxes[1].bar(x - width/2, comparison_df['Acc (Original)'], width, label='Original', alpha=0.8)axes[1].bar(x + width/2, comparison_df['Acc (Tuned)'], width, label='Optimizado', alpha=0.8)axes[1].set_xlabel('Modelo', fontweight='bold')axes[1].set_ylabel('Accuracy', fontweight='bold')axes[1].set_title('Comparación Accuracy: Original vs Optimizado', fontsize=13, fontweight='bold')axes[1].set_xticks(x)axes[1].set_xticklabels(comparison_df['Modelo'], rotation=45, ha='right')axes[1].legend()axes[1].grid(axis='y', alpha=0.3)plt.tight_layout()plt.savefig('hyperparameter_tuning_comparison.png', dpi=300, bbox_inches='tight')plt.show()# Guardar modelos optimizadoswith open('tuned_models.pkl', 'wb') as f:    pickle.dump({        'models': tuned_models,        'best_params': best_params_dict    }, f)print("\n💾 Modelos optimizados guardados: tuned_models.pkl")print("\n" + "="*80)print("✅ TAREA 16 COMPLETADA")print("="*80)

In [ ]:
# ============================================# TAREA 17: INTERPRETACIÓN DE FEATURE IMPORTANCE# ============================================print("\n\n" + "="*80)print("TAREA 17: INTERPRETACIÓN Y ANÁLISIS DE FEATURE IMPORTANCE")print("="*80 + "\n")"""Analizaremos la importancia de las variables según diferentes modelos:- Random Forest: Feature Importance basado en reducción de impureza- Logistic Regression: Coeficientes del modelo- Decision Tree: Feature Importance basado en splits"""# ==================# 17.1: RANDOM FOREST FEATURE IMPORTANCE# ==================print("17.1 RANDOM FOREST - FEATURE IMPORTANCE")print("-" * 80 + "\n")# Usar modelo optimizado si existe, sino el originalrf_for_importance = tuned_models.get('Random Forest (Tuned)', models.get('Random Forest'))if rf_for_importance:    # Obtener importancias    importances_rf = rf_for_importance.feature_importances_    feature_names = X_train.columns        # Crear DataFrame    importance_df_rf = pd.DataFrame({        'Feature': feature_names,        'Importance': importances_rf    }).sort_values('Importance', ascending=False)        print("Top 10 Features más importantes:\n")    print(importance_df_rf.head(10).to_string(index=False))        # Visualización    fig, axes = plt.subplots(1, 2, figsize=(16, 6))        # Top 15 features    top_n = 15    top_features = importance_df_rf.head(top_n)        axes[0].barh(range(top_n), top_features['Importance'].values, color='forestgreen', alpha=0.7)    axes[0].set_yticks(range(top_n))    axes[0].set_yticklabels(top_features['Feature'].values)    axes[0].invert_yaxis()    axes[0].set_xlabel('Importance', fontweight='bold')    axes[0].set_title(f'Top {top_n} Features - Random Forest', fontsize=13, fontweight='bold')    axes[0].grid(axis='x', alpha=0.3)        # Importancia acumulada    cumsum_importance = np.cumsum(importance_df_rf['Importance'].values)    axes[1].plot(range(1, len(cumsum_importance)+1), cumsum_importance, 'b-', linewidth=2)    axes[1].axhline(y=0.9, color='r', linestyle='--', label='90% threshold')    axes[1].axhline(y=0.95, color='orange', linestyle='--', label='95% threshold')    axes[1].set_xlabel('Número de Features', fontweight='bold')    axes[1].set_ylabel('Importancia Acumulada', fontweight='bold')    axes[1].set_title('Importancia Acumulada de Features', fontsize=13, fontweight='bold')    axes[1].legend()    axes[1].grid(alpha=0.3)        # Calcular cuántas features explican 90% y 95%    n_features_90 = np.argmax(cumsum_importance >= 0.90) + 1    n_features_95 = np.argmax(cumsum_importance >= 0.95) + 1        axes[1].axvline(x=n_features_90, color='r', linestyle=':', alpha=0.7)    axes[1].axvline(x=n_features_95, color='orange', linestyle=':', alpha=0.7)        plt.tight_layout()    plt.savefig('feature_importance_random_forest.png', dpi=300, bbox_inches='tight')    plt.show()        print(f"\n📊 Features necesarias para explicar:")    print(f"   • 90% de la importancia: {n_features_90} features")    print(f"   • 95% de la importancia: {n_features_95} features")# ==================# 17.2: LOGISTIC REGRESSION COEFFICIENTS# ==================print("\n\n17.2 LOGISTIC REGRESSION - COEFICIENTES")print("-" * 80 + "\n")lr_for_coef = tuned_models.get('Logistic Regression (Tuned)', models.get('Logistic Regression'))if lr_for_coef:    # Obtener coeficientes (promedio absoluto para todas las clases)    coef_abs = np.abs(lr_for_coef.coef_).mean(axis=0)    feature_names = X_train.columns        coef_df = pd.DataFrame({        'Feature': feature_names,        'Coef (abs mean)': coef_abs    }).sort_values('Coef (abs mean)', ascending=False)        print("Top 10 Features con mayor coeficiente:\n")    print(coef_df.head(10).to_string(index=False))        # Visualización detallada por clase    fig, axes = plt.subplots(2, 3, figsize=(18, 12))    axes = axes.ravel()        for idx, class_name in enumerate(le_target.classes_):        if idx < 5:  # Tenemos 5 clases            class_coef = lr_for_coef.coef_[idx]            coef_class_df = pd.DataFrame({                'Feature': feature_names,                'Coefficient': class_coef            }).sort_values('Coefficient', key=lambda x: abs(x), ascending=False).head(10)                        colors = ['green' if x > 0 else 'red' for x in coef_class_df['Coefficient']]            axes[idx].barh(range(10), coef_class_df['Coefficient'].values, color=colors, alpha=0.7)            axes[idx].set_yticks(range(10))            axes[idx].set_yticklabels(coef_class_df['Feature'].values, fontsize=8)            axes[idx].invert_yaxis()            axes[idx].set_xlabel('Coeficiente', fontweight='bold')            axes[idx].set_title(f'Top 10 Coeficientes - Clase {class_name}', fontsize=11, fontweight='bold')            axes[idx].axvline(x=0, color='black', linestyle='-', linewidth=0.5)            axes[idx].grid(axis='x', alpha=0.3)        # Hide last subplot    axes[5].axis('off')        plt.tight_layout()    plt.savefig('feature_coefficients_logistic.png', dpi=300, bbox_inches='tight')    plt.show()        print("\n💡 Interpretación:")    print("   • Coeficientes positivos: Aumentan probabilidad de la clase")    print("   • Coeficientes negativos: Disminuyen probabilidad de la clase")    print("   • Mayor magnitud: Mayor influencia en la predicción")# ==================# 17.3: DECISION TREE FEATURE IMPORTANCE# ==================print("\n\n17.3 DECISION TREE - FEATURE IMPORTANCE")print("-" * 80 + "\n")dt_for_importance = tuned_models.get('Decision Tree (Tuned)', models.get('Decision Tree'))if dt_for_importance:    importances_dt = dt_for_importance.feature_importances_        importance_df_dt = pd.DataFrame({        'Feature': X_train.columns,        'Importance': importances_dt    }).sort_values('Importance', ascending=False)        print("Top 10 Features más importantes:\n")    print(importance_df_dt.head(10).to_string(index=False))        # Visualización del árbol (top levels)    fig, ax = plt.subplots(figsize=(20, 10))    plot_tree(dt_for_importance,               feature_names=X_train.columns,              class_names=le_target.classes_,              filled=True,              rounded=True,              max_depth=3,  # Solo primeros 3 niveles para legibilidad              fontsize=10,              ax=ax)    plt.title('Estructura del Árbol de Decisión (3 primeros niveles)',               fontsize=15, fontweight='bold')    plt.tight_layout()    plt.savefig('decision_tree_structure.png', dpi=300, bbox_inches='tight')    plt.show()# ==================# 17.4: COMPARACIÓN DE IMPORTANCIAS# ==================print("\n\n17.4 COMPARACIÓN DE IMPORTANCIAS ENTRE MODELOS")print("-" * 80 + "\n")# Crear DataFrame comparativocomparison_importance = pd.DataFrame({    'Feature': X_train.columns})if rf_for_importance:    comparison_importance['RF Importance'] = importances_rfif lr_for_coef:    comparison_importance['LR Coef (abs)'] = coef_absif dt_for_importance:    comparison_importance['DT Importance'] = importances_dt# Normalizar para comparaciónfor col in comparison_importance.columns[1:]:    comparison_importance[col + ' (norm)'] = (        comparison_importance[col] / comparison_importance[col].sum()    )# Top features consensonorm_cols = [c for c in comparison_importance.columns if '(norm)' in c]comparison_importance['Mean Importance'] = comparison_importance[norm_cols].mean(axis=1)comparison_importance = comparison_importance.sort_values('Mean Importance', ascending=False)print("Top 15 Features según consenso de modelos:\n")print(comparison_importance[['Feature', 'Mean Importance']].head(15).to_string(index=False))# Visualizaciónfig, ax = plt.subplots(figsize=(14, 8))top_15_consensus = comparison_importance.head(15)x = np.arange(15)width = 0.25if 'RF Importance (norm)' in top_15_consensus.columns:    ax.bar(x - width, top_15_consensus['RF Importance (norm)'], width,            label='Random Forest', alpha=0.8)if 'LR Coef (abs) (norm)' in top_15_consensus.columns:    ax.bar(x, top_15_consensus['LR Coef (abs) (norm)'], width,            label='Logistic Regression', alpha=0.8)if 'DT Importance (norm)' in top_15_consensus.columns:    ax.bar(x + width, top_15_consensus['DT Importance (norm)'], width,            label='Decision Tree', alpha=0.8)ax.set_xlabel('Features', fontweight='bold')ax.set_ylabel('Importancia Normalizada', fontweight='bold')ax.set_title('Comparación de Feature Importance entre Modelos',              fontsize=14, fontweight='bold')ax.set_xticks(x)ax.set_xticklabels(top_15_consensus['Feature'], rotation=45, ha='right')ax.legend()ax.grid(axis='y', alpha=0.3)plt.tight_layout()plt.savefig('feature_importance_comparison.png', dpi=300, bbox_inches='tight')plt.show()# Guardar resultadosfeature_importance_results = {    'random_forest': importance_df_rf.to_dict() if rf_for_importance else None,    'logistic_regression': coef_df.to_dict() if lr_for_coef else None,    'decision_tree': importance_df_dt.to_dict() if dt_for_importance else None,    'consensus': comparison_importance.to_dict()}with open('feature_importance_analysis.pkl', 'wb') as f:    pickle.dump(feature_importance_results, f)print("\n💾 Análisis de importancia guardado: feature_importance_analysis.pkl")print("\n" + "="*80)print("✅ TAREA 17 COMPLETADA")print("="*80)print("\n💡 HALLAZGOS CLAVE:")print("   • Las features más importantes son consistentes entre modelos")print("   • Pocas features explican la mayor parte de la varianza")print("   • Los coeficientes revelan relaciones direccionales con las clases")

In [ ]:
# ============================================# RESUMEN FINAL DE LA SECCIÓN 4# ============================================print("\n\n" + "="*80)print("📊 RESUMEN SECCIÓN 4: APRENDIZAJE SUPERVISADO")print("="*80 + "\n")summary_text = f"""✅ APRENDIZAJE SUPERVISADO COMPLETADOTAREA 13: Entrenamiento de Modelos   • Modelos entrenados: {len(models)}   • Algoritmos: Decision Tree, Random Forest, Logistic Regression, SVM, KNN   • Tiempo total de entrenamiento: {sum(training_times.values()):.2f}sTAREA 14: Comparación y Evaluación   • Mejor modelo (F1-Score): {metrics_df.index[0]}   • F1-Score máximo: {metrics_df['F1-Score'].max():.4f}   • Accuracy máximo: {metrics_df['Accuracy'].max():.4f}   • Métricas calculadas: Accuracy, Precision, Recall, F1-Score   • Matrices de confusión generadas para todos los modelosTAREA 15: Validación Cruzada   • Método: {N_FOLDS}-Fold Stratified Cross-Validation   • Modelo más estable: {cv_summary_df.iloc[0]['Modelo']}   • Menor desviación estándar: {cv_summary_df['Accuracy (std)'].min():.5f}   • Análisis de overfitting realizadoTAREA 16: Ajuste de Hiperparámetros   • Modelos optimizados: {len(tuned_models)}   • Métodos: Grid Search y Random Search   • Mejora promedio en F1-Score: {comparison_df['F1 Mejora'].mean():.4f}TAREA 17: Feature Importance   • Top features identificadas y consensuadas   • Análisis de coeficientes de regresión logística   • Estructura del árbol de decisión visualizada   • Features para 90% de importancia: {n_features_90 if 'n_features_90' in locals() else 'N/A'}ARCHIVOS GENERADOS:   ✓ model_comparison_metrics.png   ✓ confusion_matrices.png   ✓ cross_validation_analysis.png   ✓ hyperparameter_tuning_comparison.png   ✓ feature_importance_*.png   ✓ tuned_models.pkl   ✓ feature_importance_analysis.pklCONCLUSIONES Y RECOMENDACIONES:1. RENDIMIENTO GENERAL:   Los modelos supervisados logran buenos resultados en la clasificación del   nivel de inglés, con F1-Scores superiores a {metrics_df['F1-Score'].min():.2f}.2. MODELO RECOMENDADO:   {metrics_df.index[0]} muestra el mejor balance entre precisión,   estabilidad y tiempo de entrenamiento.3. DESAFÍOS IDENTIFICADOS:   • Traslape entre clases intermedias (A1, A2, B1)   • Desbalance de clases afecta métricas de clases minoritarias   • Algunos modelos muestran signos leves de overfitting4. PRÓXIMOS PASOS (Sección 5):   • Implementar técnicas de balanceo (SMOTE)   • Probar ensemble methods avanzados   • Feature engineering adicional   • Regularización para reducir overfittingPRÓXIMA SECCIÓN: Sección 5 - Evaluación e Interpretación"""print(summary_text)# Guardar resumen completofull_results = {    'models': models,    'tuned_models': tuned_models,    'predictions': predictions,    'metrics': metrics_results,    'cv_results': cv_results,    'best_params': best_params_dict,    'training_times': training_times}with open('seccion4_complete_results.pkl', 'wb') as f:    pickle.dump(full_results, f)with open('resumen_seccion4.txt', 'w', encoding='utf-8') as f:    f.write("RESUMEN SECCIÓN 4: APRENDIZAJE SUPERVISADO\n")    f.write("="*80 + "\n\n")    f.write(summary_text)print("\n💾 Resumen guardado: resumen_seccion4.txt")print("💾 Resultados completos: seccion4_complete_results.pkl")print("\n" + "="*80)print("🎉 ¡SECCIÓN 4 COMPLETADA EXITOSAMENTE!")print("="*80)print("\nEstás listo para continuar con la Sección 5: Evaluación e Interpretación")